# **Проект исследования рекламных кампаний Сигма - тестовое в Яндекс Директ**

## **Цель исследования**

Рекламодатель Сигма жалуется на постоянный рост СРА по своим рекламным кампаниям в период с июня по сентябрь 2022 года.

Задача исследования - проанализировать данные с целью:
- выяснить причину растущего СРА
- дать рекомендации по снижению СРА до целевого значения 380 руб..

### **Описание столбцов данных:**

- Client – название клиента
- Device – тип устройства
- Category - категория рекламируемого товара ` - не было в описании данных, но есть в файле`
- Type – тип канала ` - есть в описании данных, но нету в файле!`
- Place – тип площадки (только для Type: Direct)
- BannerType – тип объявления
- QueryType – тип запроса (только для Type: Direct , только поиск)
- TargetingType – тип таргетинга (только для Type: Direct)
- Clicks - количество кликов по объявлению ` - не было в описании данных, но есть в файле`
- Cost - расходы рекламодателя
- Conversions – целевые визиты (только Директ)

### **План исследования**

Работать будем в следующем порядке:
- проверка и преобразование исходных данных
- оценка общей динамики СРА
- оценка динамики СРА с разбивкой по устройствам, каналам, площадкам и так далее
- выявление проблем
- подготовка рекомендаций

In [31]:
# перед началом работы импортируем библиотеки, которые могут понадобиться в работе над проектом

import pandas as pd
import math
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats as stats
from urllib.parse import urlencode 
import re
import requests
from calendar import day_name
import plotly
import plotly.graph_objects as go
import plotly.express as px

%matplotlib inline

In [87]:
# будем отображать цифры без экспоненты и только с 2 знаками после запятой
pd.options.display.float_format ='{:,.2f}'.format

## **Импорт данных**

In [2]:
# используя конструкцию try-ecxept для возможности открыть локально и с внешнего источника
try:
    df = pd.read_excel(r'C:\Users\rosto\OneDrive\Документы\Аналитика\data_for_Sigma.xlsx')
    
except:

    # используем api
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    public_key = 'https://disk.yandex.ru/i/HAryA9iCXncGdw'
    # получаем url
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    # загружаем файл в df
    download_response = requests.get(download_url)
    mark_events = pd.read_excel(download_url)

In [77]:
df.head()

,Client,Month,Category,Device,Place,BannerType,QueryType (search only),TargetingType,Shows,Clicks,Cost,Conversions
0,Псилон,2021-04-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Desktop,Networks,cpc_video,none,Phrase,0,0,0.00,0
1,Псилон,2021-04-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Desktop,Networks,cpc_video,none,Retargeting,5949,41,"1,666.84",0
2,Псилон,2021-04-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Mobile,Networks,cpc_video,none,Retargeting,191,6,40.64,0
3,Гамма,2021-04-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Desktop,Networks,image_ad,none,Retargeting,227,1,45.75,0
4,Гамма,2021-04-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Mobile,Networks,image_ad,none,Retargeting,18,0,0.00,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10478 entries, 0 to 10477
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Client                   10478 non-null  object        
 1   Month                    10478 non-null  datetime64[ns]
 2   Category                 10478 non-null  object        
 3   Device                   10478 non-null  object        
 4   Place                    10478 non-null  object        
 5   BannerType               10478 non-null  object        
 6   QueryType (search only)  10478 non-null  object        
 7   TargetingType            10478 non-null  object        
 8   Shows                    10478 non-null  int64         
 9   Clicks                   10478 non-null  int64         
 10  Cost                     10478 non-null  float64       
 11  Conversions              10478 non-null  int64         
dtypes: datetime64[ns](1), float64(1)

Данные в столбцах имеют корректный формат, пропусков в данных нет, можно приступать к дальнейшему исследованию.

## *Функция группировки данных с вычислением СРА*

In [100]:
def group_df(df, column):
    # сгруппируем количество показов, кликов, СТР и стоимости по устройствам
    grouped_df = df.groupby(column).agg({'Clicks': ['sum'],
                                             'Cost': ['sum'], 'Conversions': ['sum']}).reset_index()
    grouped_df.columns = [column, 'Кликов', 'Расход', 'Конверсий']
    grouped_df['CPA'] = grouped_df['Расход'] / grouped_df['Конверсий']
    grouped_df['CPA'] = grouped_df['CPA'].fillna(grouped_df['Расход'])
        
    #for col in ('Страниц просмотрено', '% откзов', 'Ср глубина просмотра', 'Ср сессия в сек'):
        #grouped_device[col] = round((grouped_device[col]), 2)
        
    return(grouped_df)

In [57]:
def group_df_2param(df, column1, column2):
    # сгруппируем количество показов, кликов, СТР и стоимости по устройствам
    grouped_df = df.groupby([column1, column2]).agg({'Clicks': ['sum'], 'Cost': ['sum'], 'Conversions': ['sum']}).reset_index()
    grouped_df.columns = [column1, column2, 'Кликов', 'Расход', 'Конверсий']
    grouped_df['CPA'] = grouped_df['Расход'] / grouped_df['Конверсий']
    
    fig = px.line(grouped_df, 
             x=column1, 
             y="CPA", 
             color=column2,
             title= f"Распределение CPA по {column2}")
    
    fig.show()

    #return()

# **Исследовательский анализ даных** 

In [18]:
# выделим данные по рекламным кампаниям Сигмы в отдельный датафрейм

In [20]:
df_sigma = df.query('Client == "Сигма"')

In [45]:
# посмотрим на динамику СРА по этому клиенту для того, что бы убедиться что проблема имеет место быть

In [48]:
fig = px.line(group_df(df_sigma, 'Month'), 
             x='Month', 
             y="CPA")
fig.show()

В начале рассматриваемого периода видим большой всплеск СРА до 870 рублей в июне 2021 года, затем с августа по март СРА был на уровне 400-470 рублей, в апреле и мае он значительно упал до 230 рублей, и начиная с июня 2022 он показывает рост вплоть до уровня 607 руб в сентябре 2022 года.

Очевидно, рост последних месяцев присутствует.

## **Анализ CPA по рекламируемым категориям** 

In [101]:
# посмотрим на распределение CPA по устройствам 
group_df(df_sigma, 'Category')

,Category,Кликов,Расход,Конверсий,CPA
0,ВКЛАДЫ,2841713,"17,193,777.71",169193,101.62
1,ДЕБЕТОВЫЕ КАРТЫ,948381,"18,432,554.96",55468,332.31
2,ИНВЕСТИЦИИ,238510,"2,926,887.42",0,inf
3,ИПОТЕЧНОЕ КРЕДИТОВАНИЕ,2918663,"60,663,542.76",66380,913.88
4,КРЕДИТНЫЕ КАРТЫ,1857598,"33,399,689.09",162414,205.65
5,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,6207629,"123,396,193.39",427178,288.86
6,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,3742891,"130,064,162.45",23510,"5,532.29"


Видим три рекламируемые категории, где величина СРА выше целевой:
- ИНВЕСТИЦИИ, где при расходах почти в 3 млн рублей получили 0 конверсий
- ИПОТЕЧНОЕ КРЕДИТОВАНИЕ, где СРА 913,9 рублей
- РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ, где СРА 5,5 тысячи рублей

Маловероятно, что у Инвестиций при 238 тысячах кликов в действительности не было ни одной конверсии, поэтому предположу, что для Инвестиций не была настроена цель, и поэтому не отображается ни одного ее выполнения. Что бы попробовать посчитать СРА для Инвестиций попробуем посчитать, ск



In [94]:
group_df_2param(df_sigma, 'Month', 'Category')

Категория РКО постоянно показывает очень высокую стоимость СРА, после периода некоторого снижения в начале 2022 года с сентября наблюдаем значительный рост.
Категория Ипотечного кредитования покавает снижение стоимости СРА, но все равно выше целевых значений

## **Анализ CPA по устройствам** 

In [78]:
# посмотрим на распределение CPA по устройствам 
group_df(df_sigma, 'Device')

,Device,Кликов,Расход,Конверсий,CPA
0,Desktop,6332515,"185,685,337.80",355627,522.14
1,Mobile,12422870,"200,391,469.98",548516,365.33


Видим, что СРА на мобильных устройствах укладывается в целевую величину, а на десктопах - значительно ее превышает. Посмотрим на динамику этого показателя

In [58]:
group_df_2param(df_sigma, 'Month', 'Device')

На графике видим, что СРА для десктопов всегда было выше, чем для мобильных устройств, общая динамика по обоим девайсам примерно одинаковая - с июня 2022 года наблюдается рост.
Также на этом графике видно, по какому устройству был всплеск СРА в июне 2021 года.

Вывод - проблема роста СРА не в типе устройства.

## **Анализ CPA по типу площадки** 

In [80]:
# посмотрим на распределение CPA по типу площадки 
group_df(df_sigma, 'Place')

,Place,Кликов,Расход,Конверсий,CPA
0,Networks,10021953,"125,988,149.40",251480,500.99
1,Search,8733432,"260,088,658.37",652663,398.50


Размещение в рекламной сети за весь период значительно дороже, чем на поиске. Посмотрим на динамику этих мест размещения

In [59]:
group_df_2param(df_sigma, 'Month', 'Place')

Видим, что до марта 2022 года показатели обоих видов площадок были примерно одинаковы, после этого месяца СРА в поиске медленно снижался, в то время как СРА в рекламной сети сначала упал до 60-70 рублей, а с июня показывает рост аж до 2382 рублей за целевое действие в сентябре 2022 года.

Разделим датафрейм по типу площадки и посмотрим на них по отдельности.

In [85]:
df_sigma_netw = df_sigma.query('Place == "Networks"')
df_sigma_search = df_sigma.query('Place != "Networks"')

## **Анализ CPA по типу объявления** 

In [84]:
# посмотрим на распределение CPA по типу объявления в сетях
group_df(df_sigma_netw, 'BannerType')

,BannerType,Кликов,Расход,Конверсий,CPA
0,cpc_video,92,"1,861.68",0,inf
1,cpm_banner,0,0.00,0,NaN
2,image_ad,202665,"2,119,172.47",376,"5,636.10"
3,text,9819196,"123,867,115.25",251104,493.29


В рекламе в рекламной сети видим очень высокую, 5,6 тыс рублей, СРА для объявления "с изображениями"

In [88]:
# посмотрим на распределение CPA по типу объявления в поиске
group_df(df_sigma_search, 'BannerType')

,BannerType,Кликов,Расход,Конверсий,CPA
0,mcbanner,1167,"650,713.81",20,"32,535.69"
1,text,8732265,"259,437,944.56",652643,397.52


Для поиска видим огромную величину СРА - более 32 тыс рублей, для медийно-контекстных баннеров

In [92]:
df_sigma_netw.query('BannerType == "image_ad"').sort_values(by = 'Cost', ascending = False).head(10)

,Client,Month,Category,Device,Place,BannerType,QueryType (search only),TargetingType,Shows,Clicks,Cost,Conversions
3024,Сигма,2021-12-01,ПОТРЕБИТЕЛЬСКОЕ КРЕДИТОВАНИЕ,Mobile,Networks,image_ad,none,Phrase,2862511,24042,"208,646.61",115
826,Сигма,2021-12-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Mobile,Networks,image_ad,none,Phrase,2710908,23706,"169,776.77",0
1649,Сигма,2022-08-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Mobile,Networks,image_ad,none,Phrase,1784312,12391,"160,523.52",0
827,Сигма,2021-12-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Mobile,Networks,image_ad,none,Retargeting,1602488,12957,"142,875.04",0
922,Сигма,2022-01-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Mobile,Networks,image_ad,none,Phrase,2152820,26200,"107,257.94",7
1650,Сигма,2022-08-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Mobile,Networks,image_ad,none,Retargeting,472245,3589,"104,250.95",1
728,Сигма,2021-11-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Mobile,Networks,image_ad,none,Retargeting,992859,10114,"95,651.22",0
1106,Сигма,2022-03-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Mobile,Networks,image_ad,none,Retargeting,915625,8705,"93,630.12",7
1531,Сигма,2022-07-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Mobile,Networks,image_ad,none,Phrase,1939401,10362,"74,260.37",0
1779,Сигма,2022-09-01,РАСЧЕТНО-КАССОВОЕ ОБСЛУЖИВАНИЕ,Mobile,Networks,image_ad,none,Phrase,451711,3444,"58,342.71",0


## **Анализ CPA по типу запроса** 

In [82]:
# посмотрим на распределение CPA по типу запроса 
group_df(df_sigma, 'QueryType (search only)')

,QueryType (search only),Кликов,Расход,Конверсий,CPA
0,Brand,6717423,"136,956,140.12",542207,252.59
1,Competitor,606502,"33,948,431.43",28003,"1,212.31"
2,Other,1409507,"89,184,086.82",82453,"1,081.64"
3,none,10021953,"125,988,149.40",251480,500.99


In [64]:
group_df_2param(df_sigma, 'Month', 'QueryType (search only)')

In [66]:
# посмотрим на распределение CPA по типу объявления 
group_df(df_sigma, 'TargetingType')

,TargetingType,Кликов,Расход,Конверсий,CPA
0,Autotargeting,35167,5.113016e+05,2008,254.632251
1,Phrase,17815843,3.718708e+08,887734,418.898869
2,Retargeting,904375,1.369474e+07,14401,950.957384


In [67]:
group_df_2param(df_sigma, 'Month', 'TargetingType')

# **Общий вывод исследования**

В целом по исследованным данным можно сделать следующие выводы:
- подавляющая часть трафика на сайт это платный рекламный трафик
- трафик благодаря органическому поиску дает в 10 раз меньший трафик, чем рекламный
- рекламный трафик, как мы видели и в данных Директа неравномерный по дням, есть высокие пики, есть отсутствие заходов на сайт вообще
- самые популярные кампании те, которые касаются поиска ФН, у этих же кампаний большая глубина просмотра и среднее время на сайте
- у кампаний из РСЯ высокий % отказов и как правило короткие сессии
- визитов из РСЯ в 2 раза больше, чем из платного поиска, но эти визиты очень короткие, с высоким % отказов и низкой глубиной просмотра
- есть список платформ из РСЯ, которые необходимо исключить из показов. Показов станет меньше, но их эффективность повысится
- ¾ посетителей сайта заходят и уходят с него с одной и той же страницы, проводя на ней около 1 минуты. Больше всего времени проводят на страницах аренды, меньше всего на страницах ФН во Владимире и кассы Эвотор 7-3
- самая высокая конверсия (выше 10%)  в действие клик по номеру телефона – у страниц информации по ФН. У всех остальных страниц конверсия около 1%
